In [2]:
import pandas as pd
import numpy as np
import json
import collections
import copy
import random
import math
import matplotlib.pyplot as plt
from nlp.preprocessing import (
    clean_text,
    preprocess,
    tokenize,
    preprocess_document,
    tokenize_document,
    get_stopwords, 
    lemmatization_document,
    get_canonical_words,
    spellcheck_document)
from nlp.pos_tagging import (
    get_tokens_tags
)
from nlp.utils import (
    read_json_file,
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)
from nlp.grouping import (
    get_groups,
    get_groups_size,
    get_unigram_groups,
    get_two_tokens_groups,
    get_first_token_groups,
    get_bigram_groups,
    get_first_two_groups,
    groups_frequency_sort
)
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)
from item.spellcheckeropt import SpellcheckerOpt
from item.utils import get_tokens_set
from textpp_ptbr.preprocessing import TextPreProcessing as tpp
from gensim.parsing.preprocessing import (
    strip_multiple_whitespaces,
    strip_non_alphanum,
    strip_punctuation2,
    strip_short)

In [3]:
def get_items_licitacao(file_items, include_price=True, include_dsc_unidade=True,
                          recurso_limit=5.0, area='Saúde'):

    file_recurso = '../dados/licitacao_vlr_recurso_funcao.csv'
    data_recurso = pd.read_csv(file_recurso, sep=';')
    licitacoes = data_recurso.loc[(data_recurso['nom_funcao'] == area) & \
                                  (data_recurso['proporcao_vlr'] >= recurso_limit)]
    seq_dim_licitacao_list = list(licitacoes['seq_dim_licitacao'])

    data = pd.read_csv(file_items, sep=';')
    data = data.loc[data['seq_dim_licitacao'].isin(seq_dim_licitacao_list)]

    if include_price and include_dsc_unidade:
        items = data[['nom_item', 'seq_dim_licitacao', 'vlr_unitario_homologado', 'dsc_unidade_medida']].values.tolist()
    elif include_price:
        items = data[['nom_item', 'vlr_unitario_homologado']].values.tolist()
    else:
        items = list(data['nom_item'])

    return items


In [4]:
file = '../dados/itens_pregao_pitem_saude.csv'
items = get_items_licitacao(file)

In [5]:
len(items)

1530592

In [6]:
items[:10]

[['COLAGENASE 0,6UI/G', 1191385, 8.97, 'UNIDADE'],
 ['CAMPO OPERATÓRIO 45CMX50CM', 1007594, 161.77, 'PCT'],
 ['KIT PET', 1162895, 71.42, 'UNIDADE'],
 ['TINTA RELEVO PARA TECIDO BRAN.', 1043060, 24.45, 'CAIXA'],
 ['Termômetro clínico digital portátil para medir temperatura, com leitura digital em escala Celsius.',
  1021726,
  10.9,
  'UNIDADE'],
 ['Diclofenaco de Sodio - 75mg /3ml', 1181254, 0.503, 'AMPOLA'],
 ['017308 PANFLETO INFORMATIVO PARA DIVUGACAO DE E', 1134250, 0.34, 'UNIDADE'],
 ['Bota de segurança, cano curto, tipo ipermeavedl, de uso proficional, confeccionada em policloreto de vinila (pvc) injetado em uma so peça. Cano Longo:325 mmn  39. Solado Desenho anti-derrapante, de facil limpeza/higienização.',
  999848,
  34.5,
  'Caixa 1 Par'],
 ['CIPROFLOXACINO, 500 MG', 1186010, 0.17, 'comprimido'],
 ['Serviço de Recapagem de Pneus Misto 1100 x 22', 1012681, 670.0, 'Unidade']]

In [6]:
right_word = read_json_file('../dados/palavras/right_words_nilc.json')

In [7]:
items_descriptions = []
stopwords_ = get_stopwords()
relevant_stopwords = {'para', 'com', 'nao', 'mais', 'muito', 'so', 'sem', 'mesmo', 'mesma', 'ha', 'haja', 'hajam', 'houver', 'houvera', 'seja', 'sejam', 'fosse', 'fossem', 'forem', 'sera', 'serao', 'seria', 'seriam', 'tem', 'tinha', 'teve', 'tinham', 'tenha', 'tiver', 'tiverem', 'tera', 'terao', 'teria', 'teriam', 'uma', 'mais', 'entre'}
stopwords_ = stopwords_ - relevant_stopwords
canonical_form, word_class = get_canonical_words()

In [8]:
stopwords_

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'ate',
 'até',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'era',
 'eram',
 'eramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estao',
 'estas',
 'estava',
 'estavam',
 'estavamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiveramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivessemos',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'foramos',
 'formos',
 'fossemos',
 'fui',
 'fôramos',
 'fôssemos',
 'hajamos',
 'hao',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houveram',
 'houveramos',
 'houverao',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houveriamos',
 'houv

# Preprocessing items descriptions

In [9]:
for item in items:
    description = item[0]
    licitacao = item[1]
    price = item[2]
    dsc_unidade = item[3]
    if type(dsc_unidade) == str:
        dsc_unidade = tpp.remove_accents(dsc_unidade.lower())
    elif math.isnan(dsc_unidade):
        dsc_unidade = ""
    doc = preprocess_document(description, remove_numbers=False, stopwords=stopwords_)
    doc = tokenize_document(doc)
    doc = spellcheck_document(doc, right_word)
    doc = lemmatization_document(doc, canonical_form)
    items_descriptions.append((doc, licitacao, price, dsc_unidade, description))

In [10]:
len(items_descriptions)

1530592

In [ ]:
items_preprocessed = []

for doc in items_descriptions:
    flag = False
    for tok in doc[0]:
        if 'servico' in tok or 'prestacao' in tok or 'servicos' in tok or 'prestacoes' in tok:
            flag = True
            break
    if 'servico' in doc[3] or 'prestacao' in doc[3] or 'servicos' in doc[3] or 'prestacoes' in doc[3]:
        flag = True
    if flag == False:
        items_preprocessed.append(doc[0])

In [ ]:
items_preprocessed[:10]

In [ ]:
len(items_preprocessed)

In [ ]:
with open("../dados/items_preprocessed.json", "w") as jfile:
    json.dump(items_preprocessed, jfile)

In [ ]:
licitacao_items = collections.defaultdict(list)

for d in items_descriptions:
    licitacao_items[d[1]].append(d[0])

In [ ]:
len(licitacao_items)

In [ ]:
with open("../dados/licitacao_items_preprocessed.json", "w") as jfile:
    json.dump(licitacao_items, jfile)

In [ ]:
items_descriptions[:100]

In [ ]:
itemlist = ItemList()
itemlist.load_items_from_list(items_descriptions)

In [ ]:
itemlist.save_items('../dados/items_preprocessed_v3')

In [13]:
itemlist = ItemList()
itemlist.load_items_from_file('../dados/items_preprocessed_v3.zip', original=True)

In [14]:
len(itemlist.items_list)

1530512

In [15]:
servicos = ItemList()
newitems = ItemList()

for item in itemlist.items_list:
    item_dict = item.get_item_dict()
    flag = False
    for tok in item_dict['palavras']:
        if 'servico' in tok or 'prestacao' in tok or 'servicos' in tok or 'prestacoes' in tok:
            flag = True
            break
    if 'servico' in item_dict['dsc_unidade_medida'] or 'prestacao' in item_dict['dsc_unidade_medida'] or \
       'servicos' in item_dict['dsc_unidade_medida'] or 'prestacoes' in item_dict['dsc_unidade_medida']:
        flag = True
    if flag:
        servicos.items_list.append(item)
    else:
        newitems.items_list.append(item)

In [16]:
len(servicos.items_list)

40708

In [17]:
len(newitems.items_list)

1489804

In [18]:
servicos.save_items('../dados/items_preprocessed_v3_servicos')

In [19]:
newitems.save_items('../dados/items_preprocessed_v3')

In [20]:
newitems.items_list[0].print_item()

{'palavras': ['colagenase'], 'unidades_medida': ['ui'], 'números': ['0', '6'], 'cores': [], 'materiais': [], 'tamanho': [], 'quantidade': [], 'preço': 8.97, 'dsc_unidade_medida': 'unidade', 'original': 'COLAGENASE 0,6UI/G', 'licitação': 1191385, 'original_prep': ['colagenase', '0', '6', 'ui', 'g']}


# Split train-test set

In [2]:
itemlist = ItemList()
itemlist.load_items_from_file('../dados/items_preprocessed_v3.zip', original=True)

In [3]:
len(itemlist.items_list)

1489804

In [6]:
licitacoes_set = set()

for item in itemlist.items_list:
    item_dict = item.get_item_dict()
    licitacoes_set.add(item_dict['licitação'])

In [14]:
random.seed(999)
train_set = set(random.sample(list(licitacoes_set), int(0.8*len(licitacoes_set))))
test_set = licitacoes_set - train_set

In [15]:
print(len(train_set))
print(len(test_set))
print(len(licitacoes_set))

24466
6117
30583


In [17]:
train = ItemList()
test = ItemList()

for item in itemlist.items_list:
    item_dict = item.get_item_dict()
    if item_dict['licitação'] in train_set:
        train.items_list.append(item)
    else:
        test.items_list.append(item)

In [18]:
len(train.items_list)

1195143

In [19]:
len(test.items_list)

294661

In [21]:
train.save_items('../dados/items_preprocessed_v3_train')

In [22]:
test.save_items('../dados/items_preprocessed_v3_test')